Coursework - Task 3

In [1]:
# Importing the Librarys
import pandas as pd
import numpy as np


# Importing and reading the data from the csv file
np.random.seed(42)
data = pd.read_csv('fashion-mnist_train.csv')
print(data.shape)
data = data.sample(frac=1)
print(data[['label']].groupby('label').size().reset_index())

# Converting the clothes categories to one-hot encoded vectors and prepares the training and testing
one_hot = pd.get_dummies(data['label'].unique())
one_hot['label'] = one_hot.index

data = pd.merge(data,one_hot)
data = data.sample(frac=1)
data_train = data
data_test = pd.read_csv('fashion-mnist_test.csv')
data_test = pd.merge(data_test,one_hot)
data_train.drop('label',axis=1,inplace=True)
data_test.drop('label',axis=1,inplace=True)

# Creating the training and testing set
X_train = np.array(data_train.drop([0,1,2,3,4,5,6,7,8,9],axis=1).values)/255
y_train = np.array(data_train[[0,1,2,3,4,5,6,7,8,9]].values)
X_test = np.array(data_test.drop([0,1,2,3,4,5,6,7,8,9],axis=1).values)/255
y_test = np.array(data_test[[0,1,2,3,4,5,6,7,8,9]].values)

X_train = X_train.T
y_train = y_train.T
print(X_train.shape)
print(y_train.shape)
X_test = X_test.T
y_test = y_test.T

(60000, 785)
   label     0
0      0  6000
1      1  6000
2      2  6000
3      3  6000
4      4  6000
5      5  6000
6      6  6000
7      7  6000
8      8  6000
9      9  6000
(784, 60000)
(10, 60000)


Implementing Sigmoid, Relu and Softmax layers

In [2]:
# Initializing the Weight Matrices
def sigmoid(x):
    y = 1./(1+np.exp(-x))
    return y

def sigmoid_backpass(x):
    s = 1/(1+np.exp(-x))
    dx = s * (1-s)
    return dx

def relu(x):
    y = np.maximum(0,x)
    return y

def relu_backpass(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def softmax(x):
    e_x = np.exp(x - np.max(x))
    total = e_x.sum(axis=0)
    return (e_x / total) 

•	Implement an optimizer (e.g. Adam) 

In [3]:
# Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr= 0.1)

NameError: name 'nn' is not defined

Implementing a fully parameterizable network

In [4]:
# Initialize a network
def initialize_network(inputs, hidden, outputs):
    network = list()
    #Define hidden layers of the network. Weights are randomised
    hidden_layer = [{'weights':[random() for i in range(inputs + 1)]} for i in range(hidden)]
    #Add the defined hidden layer to the network. More layers can easily be added
    network.append(hidden_layer)
    #Define the output layer of the network. Weights are randomised
    output_layer = [{'weights':[random() for i in range(hidden + 1)]} for i in range(outputs)]
    #Add the defined hidden layer to the network.
    network.append(output_layer)
    return network
 
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = 0
    # Neuron activation is calculated as the weighted sum of inputs
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    #Still have to include the weight that has now input to multiply with
    activation += weights[-1]
    return activation

# Return the derivative of an output
def transfer_derivative(x):
    y = x * (1.0 - x)
    return y

# Forward propagate inputs to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        #For every neuron in a layer, calculate the activation and active it with an activation function to get new inputs
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = sigmoid(activation)
            #neuron['output'] = relu(activation)
            #neuron['output'] = softmax(activation)
            new_inputs.append(neuron['output'])
        # replace old input with new input for the next layer in the network
        inputs = new_inputs
    return inputs

Training using Backpropagation

In [5]:
import random
random.seed(42)
w1 = np.random.rand(128,784)/np.sqrt(784)
b0 = np.zeros((128,1))/np.sqrt(784)
w2 = np.random.rand(10,128)/np.sqrt(128)
b1 = np.zeros((10,1))/np.sqrt(128)
loss=[]
batches = 1000

lr = 0.1
batch_size = 200
beta = 0.9
count = 0
epochs = 500

loss_weight_dict = {

}

# For every one cycle of training the neural network
for i in range(epochs):
    permutation = np.random.permutation(X_train.shape[1])
    X_train_shuffled = X_train[:, permutation]
    Y_train_shuffled = y_train[:, permutation]
    
    for j in range(batches):
        
        begin = j * batch_size
        end = min(begin + batch_size, X_train.shape[1] - 1)
        if begin>end:
            continue
        #optimizer.zero_grad()
        X = X_train_shuffled[:, begin:end]
        Y = Y_train_shuffled[:, begin:end]
        m_batch = end - begin
        x1 = sigmoid(w1@X+b0)
        x2 = softmax(w2@x1+b1)

        delta_2 = (x2-Y)
        delta_1 = np.multiply(w2.T@delta_2, np.multiply(x1,1-x1))
        if i==0 :
            dW1 = delta_1@X.T
            dW2 = delta_2@x1.T
            db0 = np.sum(delta_1,axis=1,keepdims=True)
            db1 = np.sum(delta_2,axis=1,keepdims=True)
        else:
            dW1_old = dW1
            dW2_old = dW2
            db0_old = db0
            db1_old = db1
            dW1 = delta_1@X.T
            dW2 = delta_2@x1.T
            db0 = np.sum(delta_1,axis=1,keepdims=True)
            db1 = np.sum(delta_2,axis=1,keepdims=True)
            
            # Using the past gradients to calculate the present gradients
            dW1 = (beta * dW1_old + (1. - beta) * dW1)
            db0 = (beta * db0_old + (1. - beta) * db0)
            dW2 = (beta * dW2_old + (1. - beta) * dW2)
            db1 = (beta * db1_old + (1. - beta) * db1)


        w1 = w1 - (1./m_batch)*(dW1)*lr
        b0 = b0 - (1./m_batch)*(db0)*(lr)
        w2 = w2 - (1./m_batch)*(dW2)*lr
        b1 = b1 - (1./m_batch)*(db1)*(lr)
    
    #optimizer.step()

    x1 = sigmoid(w1@X_train+b0)
    x2_train = softmax(w2@x1+b1)
    x2_train_df = pd.DataFrame(x2_train)
    x2_train_df = (x2_train_df == x2_train_df.max()).astype(int)
    x2_train_df = x2_train_df.transpose()
    x2_train_df = pd.merge(x2_train_df,one_hot)
    x2_train_df = x2_train_df[['label']]
    y_train_df = pd.merge(pd.DataFrame(y_train.T),one_hot)
    x2_train_df['label_actual'] = y_train_df['label']
    train_accuracy = np.sum(x2_train_df['label_actual']==x2_train_df['label'])/x2_train_df.shape[0]
    add_loss = {
        'loss' : -np.mean(np.multiply(y_train,np.log(x2_train))),
        'weight_1' : w1,
        'weight_2':w2,
        'b0' : b0,
        'b1': b1,
        'train_accuracy': train_accuracy
    }
     
    x1 = sigmoid(w1@X_test+b0)
    x2_test = softmax(w2@x1+b1)
    x2_test_df = pd.DataFrame(x2_test)
    x2_test_df = (x2_test_df == x2_test_df.max()).astype(int)
    x2_test_df = x2_test_df.transpose()
    x2_test_df = pd.merge(x2_test_df,one_hot)
    x2_test_df = x2_test_df[['label']]
    y_test_df = pd.merge(pd.DataFrame(y_test.T),one_hot)
    x2_test_df['label_actual'] = y_test_df['label']
    test_accuracy = np.sum(x2_test_df['label_actual']==x2_test_df['label'])/x2_test_df.shape[0]
    print('Test: ',i)

    # Print Accuracy
    print('Testing Accuracy:',test_accuracy)
    print('Training Accuracy:',train_accuracy)
    print('\n')
    
    add_loss['testing_loss'] = -np.mean(np.multiply(y_test,np.log(x2_test)))
    add_loss['test_accuracy'] = test_accuracy
    loss_weight_dict[count] = add_loss
    count = count + 1
    

Test:  0
Testing Accuracy: 0.2
Training Accuracy: 0.22213333333333332


Test:  1
Testing Accuracy: 0.3264
Training Accuracy: 0.1991


Test:  2
Testing Accuracy: 0.2023
Training Accuracy: 0.13721666666666665


Test:  3
Testing Accuracy: 0.2683
Training Accuracy: 0.18866666666666668


Test:  4
Testing Accuracy: 0.2405
Training Accuracy: 0.5350833333333334


Test:  5
Testing Accuracy: 0.2375
Training Accuracy: 0.5619833333333333


Test:  6
Testing Accuracy: 0.2964
Training Accuracy: 0.6157166666666667


Test:  7
Testing Accuracy: 0.305
Training Accuracy: 0.6490333333333334


Test:  8
Testing Accuracy: 0.315
Training Accuracy: 0.6442666666666667


Test:  9
Testing Accuracy: 0.3095
Training Accuracy: 0.6371666666666667


Test:  10
Testing Accuracy: 0.3176
Training Accuracy: 0.6331


Test:  11
Testing Accuracy: 0.3779
Training Accuracy: 0.6738


Test:  12
Testing Accuracy: 0.1095
Training Accuracy: 0.6487166666666667


Test:  13
Testing Accuracy: 0.2999
Training Accuracy: 0.6784166666666667


KeyboardInterrupt: 